# 1. Limpieza del texto

Guillermo Luigui Ubaldo Nieto Angarita

## 1.1 Cargar librerías

In [135]:
import pdfplumber  # Para extraer texto de PDFs
import re  # Para trabajar con expresiones regulares
import spacy  # Para procesamiento de lenguaje natural (NLP)
import unicodedata  # Para normalizar caracteres Unicode
import stanza
import nltk
import fitz  # PyMuPDF
from transformers import AutoTokenizer

In [136]:
nltk.download('punkt')
stanza.download('es')
nlp_stanza = stanza.Pipeline("es")
nlp_spacy = spacy.load("es_core_news_sm")  #cargar el modelo en español
tokenizer_bert = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2025-08-15 13:24:59 INFO: Downloaded file to C:\Users\guill\stanza_resources\resources.json
2025-08-15 13:24:59 INFO: Downloading default packages for language: es (Spanish) ...
2025-08-15 13:25:00 INFO: File exists: C:\Users\guill\stanza_resources\es\default.zip
2025-08-15 13:25:03 INFO: Finished downloading models and saved to C:\Users\guill\stanza_resources
2025-08-15 13:25:03 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-08-15 13:25:03 INFO: Downloaded file to C:\Users\guill\stanza_resources\resources.json
2025-08-15 13:25:04 INFO: Loading these models for language: es (Spanish):
| Processor    | Package           |
------------------------------------
| tokenize     | combined     

## 1.2 Cargar texto

In [137]:
pdf_path = "texts/certificado.pdf"

## 1.3 Eliminar Encabezado

In [138]:
header_pattern = r"Cámara de Comercio de Barranquilla\s*CERTIFICADO DE EXISTENCIA Y REPRESENTACION LEGAL O\s*DE INSCRIPCION DE DOCUMENTOS\.\s*Fecha de expedición:.*?\nRecibo No\..*?\nCODIGO DE VERIFICACIÓN:.*?\n"
def extract_text_without_header(pdf_path):
    extracted_text = []  # Lista donde almacenaremos el texto de cada página

    with pdfplumber.open(pdf_path) as pdf:  # Abrir el archivo PDF
        for page in pdf.pages:  # Iterar por cada página del PDF
            text = page.extract_text()  # Extraer texto de la página
            if text:  # Verificar si hay texto en la página
                text = re.sub(header_pattern, "", text, flags=re.DOTALL)  # Eliminar encabezado con regex
                print(text)
                extracted_text.append(text)  # Guardar el texto limpio en la lista

    return " ".join(extracted_text)  # Unir todas las páginas en un solo texto

In [139]:
def extract_text_without_header(pdf_path):
    ignore_texts = {
        "Cámara de Comercio de Barranquilla\nCERTIFICADO DE EXISTENCIA Y REPRESENTACION LEGAL O\nDE INSCRIPCION DE DOCUMENTOS.\nFecha de expedición: 24/10/2024 - 13:11:06\n",
        "Recibo No. 12264474, Valor: 7,900\nCODIGO DE VERIFICACIÓN: YK5CB09DFF\n",
        "Página 5 de 5\n"
    }

    remove_texts = {
        "MATRICULA NO RENOVADA",
        "Actualice su registro y evite sanciones",
        """------------------------------------------------------------------------------- 
Verifique  el  contenido  y  confiabilidad  de  este  certificado,  ingresando a
www.camarabaq.org.co/  y digite el código, para que visualice la imagen generada
al  momento  de  su  expedición.  La  verificación  se  puede realizar de manera
ilimitada,  durante  60  días  calendario  contados  a  partir de la fecha de su
expedición.                                                                     
--------------------------------------------------------------------------------""",
        """**********************************************************************
*                                                                    *
*  ATENCION:. ESTE COMERCIANTE NO HA CUMPLIDO CON SU DEBER LEGAL      *
*            DE RENOVAR SU MATRICULA MERCANTIL.                      *
*                                                                    *
**********************************************************************"""
    }


    doc = fitz.open(pdf_path)
    clean_pages = []
    
    for page in doc:
        blocks = page.get_text("blocks")
        page_text = []
        for b in blocks:
            text = b[4]
            # Skip if text matches one of the ignore strings
            if text in ignore_texts:
                continue
            page_text.append(text.strip())
        clean_pages.append("\n".join(page_text))

    clean_text = "\n".join(clean_pages)

    for remove_text in remove_texts:
        clean_text = re.sub(re.escape(remove_text), "", clean_text, flags=re.DOTALL)
        clean_text = clean_text.lstrip("\n")

    
    clean_text = re.sub("C E R T I F I C A", "CERTIFICA", clean_text, flags=re.DOTALL)
    clean_text = re.sub(r"Página\s+\d+\s+de\s+\d+", "", clean_text)
    clean_text = re.sub(r'[ ]{2,}', ' ', clean_text)
    clean_text = re.sub(r'\n[ \t]*\n+', '\n', clean_text)
    #print(clean_text)

    return clean_text

In [140]:
raw_text = extract_text_without_header(pdf_path) 

## 1.4 Conversión a minúsculas

In [141]:
def convertToLowerCase(text):
    return text.lower()  # Convertir todo el texto a minúsculas y procesarlo con spaCy

## 1.5 Transformación UNICODE

In [142]:
def unicodeTransformation(text, method):
    return unicodedata.normalize(method, text)  # Normalizar caracteres Unicode (ej. á → a)

## 1.6 Tokenización

In [143]:
def spacy_tokenization(text):
    return [token.text for token in nlp_spacy(text)]

def nltk_tokenization(text):
    return nltk.word_tokenize(text, language='spanish')

def stanza_tokenization(text):
    doc = nlp_stanza(text)
    return [word.text for sent in doc.sentences for word in sent.words]

def bert_tokenization(text):
    return tokenizer_bert.tokenize(text)

def takenization(doc):
    tokens = [token.text for token in doc if token.is_alpha and not token.is_stop]
    return tokens

## 1.7 Eliminación de stop words

## 1.8 Lematización

## 1.9 Eliminación de elementos no desados

In [144]:
def preprocess_text(text):
    text_lower = convertToLowerCase(text) 

    text_unicode = unicodeTransformation(text_lower,  "NFKD")
    
    tokens = nltk_tokenization(text_unicode)

    return tokens

## 1.10 Executar el código

In [145]:
raw_text = extract_text_without_header(pdf_path) 

In [146]:
 # Extraer texto limpio del PDF
tokens = preprocess_text(raw_text)  # Preprocesar el texto con spaCy

print(tokens[:50])  # Mostrar los primeros 50 tokens como prueba

["''", 'la', 'matricula', 'mercantil', 'proporciona', 'seguridad', 'y', 'confianza', 'en', 'los', 'negocios', '.', 'renueve', 'su', 'matricula', 'mercantil', 'a', 'mas', 'tardar', 'el', '31', 'de', 'marzo', "''", 'con', 'fundamento', 'en', 'la', 'matrícula', 'e', 'inscripciones', 'efectuadas', 'en', 'el', 'registro', 'mercantil', ',', 'la', 'cámara', 'de', 'comercio', 'certifica', ':', 'certifica', 'nombre', ',', 'identificación', 'y', 'domicilio', 'razón']


## 1.11 Guardar el achievo como un txt

In [147]:
with open("texts/certificado_limpio.txt", "w", encoding="utf-8") as f:
    f.write(" ".join(tokens))